## Importing the required libraries

In [61]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime
from itertools import islice

In [2]:
# setting the name here to differentiate between the sent message and the response from the data
personName = 'Mitesh Puthran'

## Reading the data from file

In [3]:
# Eva.txt is the file which contains all the conversation which was downloaded from whatsapp in the form of text 
fbFile = open('Eva.txt', 'r',encoding='utf-8') 
content = fbFile.readlines()


In [56]:
# getting a glimpse of the data
content[:5]

['10/14/16, 17:14] Messages to this chat and calls are now secured with end-to-end encryption. Tap for more info.\n',
 '10/14/16, 17:14] Mitesh Puthran: Hey\n',
 '10/14/16, 17:14] Mitesh Puthran: Did you start with the assignment?\n',
 '10/14/16, 17:14] Eveee: Yeah\n',
 '10/14/16, 17:14] Mitesh Puthran: Great\n']

## Data pre-processing

In [6]:
# Writing a function to clean the data of unecessary symbols and punctuations as they try to interfere with word embeddings

def cleanMessage(message):
# Remove new lines within message
    cleanedMessage = message.replace('\n',' ').lower()
# Deal with some weird tokens
    cleanedMessage = cleanedMessage.replace("\xc2\xa0", "")
# Remove punctuation
    cleanedMessage = re.sub('([.,!?])','', cleanedMessage)
# Remove multiple spaces in message
    cleanedMessage = re.sub(' +',' ', cleanedMessage)
    return cleanedMessage

In [51]:
# function to seperate out the indiviual conversations from the file as they have different timestamps which is not required.

def getData():
    responseDictionary = {}
    myMessage, otherPersonsMessage, currentSpeaker = "","",""
    for index,lines in enumerate(content):
        rightBracket = lines.find(']') + 2
        justMessage = lines[rightBracket:]
        colon = justMessage.find(':')
        if(justMessage[:colon] == personName):
            if not myMessage:
                startMessageIndex = index - 1
            myMessage += justMessage[colon+2:]
        elif myMessage:
            for counter in range(startMessageIndex, 0, -1):
                currentLine = content[counter]
                rightBracket = currentLine.find(']') + 2
                justMessage = currentLine[rightBracket:]
                colon = justMessage.find(':')
                if not currentSpeaker:
                    currentSpeaker = justMessage[:colon]
                elif (currentSpeaker != justMessage[:colon] and otherPersonsMessage):
                    otherPersonsMessage = cleanMessage(otherPersonsMessage)
                    myMessage = cleanMessage(myMessage)
                    responseDictionary[otherPersonsMessage] = myMessage
                    break
                otherPersonsMessage = justMessage[colon+2:] + otherPersonsMessage
            myMessage, otherPersonsMessage, currentSpeaker = "","",""
    return responseDictionary

## Dictionary Data 

In [52]:
combinedDictionary = {}

In [53]:
# putting the seperated conversations in the dictionary as key value pair
print ('Getting Whatsapp Data')
combinedDictionary.update(getData())

Getting Whatsapp Data


In [64]:
# defining a function to see the dictonary data
def take(n, iterable):
    return list(islice(iterable, n))

In [67]:
n_items = take(2, combinedDictionary.items())
n_items

[('yeah ',
  "sure will let you know <media omitted> only when you think northeastern couldn't get anymore weirder "),
 ('i think so did you start ',
  'yes i did but i am getting errors while inserting data in the tables ')]

In [14]:
print('Total len of dictionary', len(combinedDictionary))

Total len of dictionary 4349


In [15]:
print('Saving conversation data dictionary')
np.save('conversationDictionary.npy', combinedDictionary)

Saving conversation data dictionary


In [17]:
conversationFile = open('conversationData.txt', 'w',encoding='utf-8')
for key,value in combinedDictionary.items():
    if (not key.strip() or not value.strip()):
            continue
    conversationFile.write(key.strip() + value.strip())

